In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project IV/Resources/LoanDataset - LoansDatasest.csv", index_col="customer_id")
df.head()

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
customer_id,,,,,,,,,,,,
1.0,22,59000,RENT,123.0,PERSONAL,C,"£35,000.00",16.02,10,Y,3,DEFAULT
2.0,21,9600,OWN,5.0,EDUCATION,A,"£1,000.00",11.14,1,NaN,2,NO DEFAULT
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,"£5,500.00",12.87,5,N,3,DEFAULT
4.0,23,65500,RENT,4.0,MEDICAL,B,"£35,000.00",15.23,10,N,2,DEFAULT
5.0,24,54400,RENT,8.0,MEDICAL,B,"£35,000.00",14.27,10,Y,4,DEFAULT


In [2]:
# Check unique values for each column
df.count()

,0
customer_age,32586
customer_income,32586
home_ownership,32586
employment_duration,31691
loan_intent,32586
loan_grade,32586
loan_amnt,32585
loan_int_rate,29470
term_years,32586
historical_default,11849


In [3]:
# Determin the number of unique values in each column.
print(df.nunique())

customer_age             62
customer_income        4299
home_ownership            4
employment_duration      36
loan_intent               6
loan_grade                5
loan_amnt               755
loan_int_rate           348
term_years               10
historical_default        2
cred_hist_length         29
Current_loan_status       2
dtype: int64


In [4]:
# check data types for each column
df.dtypes

,0
customer_age,int64
customer_income,object
home_ownership,object
employment_duration,float64
loan_intent,object
loan_grade,object
loan_amnt,object
loan_int_rate,float64
term_years,int64
historical_default,object


In [5]:
# convert the columns need to be in buckets (customer_income & loan_amnt)

# Remove commas from 'customer_income' column
df['customer_income'] = df['customer_income'].replace(",", "", regex=True)

# Remove '£' symbol from 'loan_amnt' column
df['loan_amnt'] = df['loan_amnt'].replace("£", "", regex=True)

# Replace NaN values in the column historical_default with "N"
df['historical_default'] = df['historical_default'].fillna("NEW")

# Remove all the rows have null values in any column
df = df.dropna()

# Remove commas from 'loan_amnt' column
df['loan_amnt'] = df['loan_amnt'].replace(",", "", regex=True)

# convert 'loan_amnt' to float data type first and integer due to the ".00" at the end of the numbers
df['loan_amnt'] = df['loan_amnt'].astype(float)

# convert both columns to integer
df['customer_income'] = df['customer_income'].astype(int)
df['loan_amnt'] = df['loan_amnt'].astype(int)

# print data types to check data formating change
df.dtypes

<ipython-input-5-03ed0108b50a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loan_amnt'] = df['loan_amnt'].replace(",", "", regex=True)
<ipython-input-5-03ed0108b50a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loan_amnt'] = df['loan_amnt'].astype(float)
<ipython-input-5-03ed0108b50a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

,0
customer_age,int64
customer_income,int64
home_ownership,object
employment_duration,float64
loan_intent,object
loan_grade,object
loan_amnt,int64
loan_int_rate,float64
term_years,int64
historical_default,object


In [6]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for "customer_income"
Q1_ci = df["customer_income"].quantile(0.25)
Q3_ci = df["customer_income"].quantile(0.75)
IQR_ci = Q3_ci - Q1_ci

# Print Q1, Q3, and IQR

print("Q1_ci:", Q1_ci)
print("Q3_ci:", Q3_ci)
print("IQR_ci:", IQR_ci)


Q1_ci: 39456.0
Q3_ci: 80000.0
IQR_ci: 40544.0


In [7]:
# Define the lower and upper bounds for outliers
lower_bound_ci = Q1_ci - 1.5 * IQR_ci
upper_bound_ci = Q3_ci + 1.5 * IQR_ci

# Print lower_bound and upper_bound

print("Lower Bound:", lower_bound_ci)
print("Upper Bound:", upper_bound_ci)


Lower Bound: -21360.0
Upper Bound: 140816.0


In [8]:
# Check outliers excluding lower_bound because it goes to negative
outliers_ci = df[df["customer_income"] > upper_bound_ci]
outliers_ci

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
customer_id,,,,,,,,,,,,
16.0,23,500000,MORTGAGE,7.0,DEBTCONSOLIDATION,A,30000,10.65,10,NEW,3,NO DEFAULT
21.0,25,162500,RENT,2.0,VENTURE,A,35000,7.49,10,NEW,4,NO DEFAULT
34.0,25,306000,RENT,2.0,DEBTCONSOLIDATION,B,24250,13.85,10,NEW,3,NO DEFAULT
35.0,26,300000,MORTGAGE,10.0,MEDICAL,B,7800,13.49,6,NEW,4,NO DEFAULT
43.0,26,300000,MORTGAGE,10.0,VENTURE,A,20000,7.88,10,NEW,4,NO DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...
32544.0,52,720000,MORTGAGE,1.0,PERSONAL,A,7925,8.94,2,N,30,NO DEFAULT
32545.0,58,780000,MORTGAGE,7.0,PERSONAL,A,4500,6.91,8,N,20,NO DEFAULT
32552.0,58,780000,MORTGAGE,7.0,PERSONAL,A,4500,6.91,8,NEW,19,NO DEFAULT


In [9]:
df_wo_outliers_ci = df[df["customer_income"] <= upper_bound_ci]
df_wo_outliers_ci

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
customer_id,,,,,,,,,,,,
1.0,22,59000,RENT,123.0,PERSONAL,C,35000,16.02,10,Y,3,DEFAULT
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT
4.0,23,65500,RENT,4.0,MEDICAL,B,35000,15.23,10,N,2,DEFAULT
5.0,24,54400,RENT,8.0,MEDICAL,B,35000,14.27,10,Y,4,DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...
32576.0,52,64500,RENT,0.0,EDUCATION,B,5000,11.26,2,NEW,20,NO DEFAULT
32577.0,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,7,NEW,30,NO DEFAULT
32578.0,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,4,NEW,19,NO DEFAULT


In [10]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for "loan_amnt"
Q1_la = df_wo_outliers_ci["loan_amnt"].quantile(0.25)
Q3_la = df_wo_outliers_ci["loan_amnt"].quantile(0.75)
IQR_la = Q3_la - Q1_la

# Print Q1, Q3, and IQR

print("Q1_la:", Q1_la)
print("Q3_la:", Q3_la)
print("IQR_la:", IQR_la)


Q1_la: 5000.0
Q3_la: 12000.0
IQR_la: 7000.0


In [11]:
# Define the lower and upper bounds for outliers
lower_bound_la = Q1_la - 1.5 * IQR_la
upper_bound_la = Q3_la + 1.5 * IQR_la

# Print lower_bound and upper_bound

print("Lower Bound_la:", lower_bound_la)
print("Upper Bound_la:", upper_bound_la)


Lower Bound_la: -5500.0
Upper Bound_la: 22500.0


In [12]:
# Check outliers excluding lower_bound because it goes to negative
outliers_la = df_wo_outliers_ci[df_wo_outliers_ci["loan_amnt"] > upper_bound_la]
outliers_la

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
customer_id,,,,,,,,,,,,
1.0,22,59000,RENT,123.0,PERSONAL,C,35000,16.02,10,Y,3,DEFAULT
4.0,23,65500,RENT,4.0,MEDICAL,B,35000,15.23,10,N,2,DEFAULT
5.0,24,54400,RENT,8.0,MEDICAL,B,35000,14.27,10,Y,4,DEFAULT
7.0,26,77100,RENT,8.0,EDUCATION,A,35000,12.42,10,NEW,3,NO DEFAULT
8.0,24,78956,RENT,5.0,MEDICAL,A,35000,11.11,10,NEW,4,NO DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...
32481.0,62,62050,MORTGAGE,29.0,MEDICAL,C,25000,13.43,2,Y,28,DEFAULT
32485.0,66,82000,MORTGAGE,8.0,PERSONAL,A,28000,7.90,6,N,20,NO DEFAULT
32504.0,65,93000,MORTGAGE,27.0,PERSONAL,B,25000,9.99,5,N,27,NO DEFAULT


In [13]:
df_wo_outliers_la = df_wo_outliers_ci[df_wo_outliers_ci["loan_amnt"] <= upper_bound_la]
df_wo_outliers_la

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
customer_id,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...
32575.0,52,65004,RENT,4.0,PERSONAL,D,20000,15.58,2,Y,19,DEFAULT
32576.0,52,64500,RENT,0.0,EDUCATION,B,5000,11.26,2,NEW,20,NO DEFAULT
32577.0,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,7,NEW,30,NO DEFAULT


In [14]:
df_w_filters = df_wo_outliers_la
df_w_filters

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
customer_id,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...
32575.0,52,65004,RENT,4.0,PERSONAL,D,20000,15.58,2,Y,19,DEFAULT
32576.0,52,64500,RENT,0.0,EDUCATION,B,5000,11.26,2,NEW,20,NO DEFAULT
32577.0,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,7,NEW,30,NO DEFAULT


In [15]:
# check the range of customer_income
print(df_w_filters['customer_income'].min())
print(df_w_filters['customer_income'].max())

4000
140400


In [16]:
# Define the bins (adjust the last bin to include all values above 80,000)
bins = [0, 20000, 40000, 60000, 80000, float('inf')]

# Define labels for each bin
labels = ["$0 - $20,000", "$20,001 - $40,000", "$40,001 - $60,000", "$60,001 - $80,000", "$80,001+"]

# Create a new column with categorized income
df_w_filters['income_category'] = pd.cut(df_w_filters['customer_income'], bins=bins, labels=labels, right=False)


# Print the updated DataFrame
df_w_filters.head()


<ipython-input-16-492e04d10204>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_filters['income_category'] = pd.cut(df_w_filters['customer_income'], bins=bins, labels=labels, right=False)


,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category
customer_id,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000"
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000"
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000"
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000"
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000"


In [17]:
# min and max value for loan_amount
min_value = df_w_filters['loan_amnt'].min()
max_value = df_w_filters['loan_amnt'].max()
print(min_value)
print(max_value)

500
22500


In [18]:
# Define the bins (adjust the last bin to include all values above 20,000)
bins = [0, 5000, 10000, 15000, 20000, float('inf')]

# Define labels for each bin
labels = ["$0 - $5,000", "$5,001 - $10,000", "$10,001 - $15,000", "$15,001 - $20,000", "$20,001+"]

# Create a new column with categorized income
df_w_filters['loan_category'] = pd.cut(df_w_filters['loan_amnt'], bins=bins, labels=labels, right=False)

# Print the updated DataFrame
df_w_filters.head()

<ipython-input-18-45063aed17a7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_filters['loan_category'] = pd.cut(df_w_filters['loan_amnt'], bins=bins, labels=labels, right=False)


,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category
customer_id,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000"
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000"
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000"
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000"
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000"


In [19]:
# min and max value for loan_int_rate
min_value = df_w_filters['loan_int_rate'].min()
max_value = df_w_filters['loan_int_rate'].max()
print(min_value)
print(max_value)

5.42
23.22


In [20]:
# Define the bins (adjust the last bin to include all values above 5)
bins = [5, 9, 13, 17, 21, float('inf')]

# Define labels for each bin
labels = ["5 - 9", "9.01 - 13", "13.01 - 17", "17.01 - 21", "21+"]

# Create a new column with categorized income
df_w_filters['Interest_Rate_Category(%)'] = pd.cut(df_w_filters['loan_int_rate'], bins=bins, labels=labels, right=False)

# Print the updated DataFrame
df_w_filters.head()

<ipython-input-20-df31be214d97>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_filters['Interest_Rate_Category(%)'] = pd.cut(df_w_filters['loan_int_rate'], bins=bins, labels=labels, right=False)


,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%)
customer_id,,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9


In [21]:
# Checking min and max of customer age
min_value = df_w_filters['customer_age'].min()
max_value = df_w_filters['customer_age'].max()
print(min_value)
print(max_value)

3
123


In [22]:
# checking the dataset with the filter "< 18," which is the minimum legal age to get loan.
df_check = df_w_filters[df_w_filters['customer_age'] < 18]
df_check

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%)
customer_id,,,,,,,,,,,,,,,
29393.0,6,25000,MORTGAGE,4.0,MEDICAL,A,11500,7.29,6,N,12,NO DEFAULT,"$20,001 - $40,000","$10,001 - $15,000",5 - 9
29394.0,8,47000,RENT,4.0,DEBTCONSOLIDATION,A,16500,7.90,8,N,16,DEFAULT,"$40,001 - $60,000","$15,001 - $20,000",5 - 9
29395.0,3,25000,MORTGAGE,10.0,EDUCATION,B,10000,9.62,3,N,17,NO DEFAULT,"$20,001 - $40,000","$10,001 - $15,000",9.01 - 13


In [23]:
# removing the wrong entry data in "Customer_age" column
df_w_filters = df_w_filters[df_w_filters['customer_age'] >= 18]
df_w_filters

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%)
customer_id,,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32575.0,52,65004,RENT,4.0,PERSONAL,D,20000,15.58,2,Y,19,DEFAULT,"$60,001 - $80,000","$20,001+",13.01 - 17
32576.0,52,64500,RENT,0.0,EDUCATION,B,5000,11.26,2,NEW,20,NO DEFAULT,"$60,001 - $80,000","$5,001 - $10,000",9.01 - 13
32577.0,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,7,NEW,30,NO DEFAULT,"$40,001 - $60,000","$5,001 - $10,000",13.01 - 17


In [24]:
# Checking again min and max of customer age
min_value = df_w_filters['customer_age'].min()
max_value = df_w_filters['customer_age'].max()
print(min_value)
print(max_value)

20
123


In [25]:
df_check = df_w_filters[df_w_filters['customer_age'] > 100]
df_check

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%)
customer_id,,,,,,,,,,,,,,,
576.0,123,80004,RENT,2.0,EDUCATION,A,20400,10.25,9,NEW,3,NO DEFAULT,"$80,001+","$20,001+",9.01 - 13


In [26]:
# removing the wrong entry data in "Customer_age" column
df_w_filters = df_w_filters[df_w_filters['customer_age'] <= 100]
df_w_filters

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%)
customer_id,,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32575.0,52,65004,RENT,4.0,PERSONAL,D,20000,15.58,2,Y,19,DEFAULT,"$60,001 - $80,000","$20,001+",13.01 - 17
32576.0,52,64500,RENT,0.0,EDUCATION,B,5000,11.26,2,NEW,20,NO DEFAULT,"$60,001 - $80,000","$5,001 - $10,000",9.01 - 13
32577.0,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,7,NEW,30,NO DEFAULT,"$40,001 - $60,000","$5,001 - $10,000",13.01 - 17


In [27]:
# Checking again min and max of customer age
min_value = df_w_filters['customer_age'].min()
max_value = df_w_filters['customer_age'].max()
print(min_value)
print(max_value)

20
99


In [28]:
# Define the bins
bins = [18, 25, 35, 45, 55, 65, float('inf')]

# Define labels for each bin
labels = ["18 - 24", "25 - 34", "35 - 44", "45 - 54", "55 - 64", "65+"]

# Create a new column with categorized income
df_w_filters['age_group'] = pd.cut(df_w_filters['customer_age'], bins=bins, labels=labels, right=False)

# Print the updated DataFrame
df_w_filters.head()

<ipython-input-28-067f3217790a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_filters['age_group'] = pd.cut(df_w_filters['customer_age'], bins=bins, labels=labels, right=False)


,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%),age_group
customer_id,,,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13,18 - 24
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13,25 - 34
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17,18 - 24
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24


In [29]:
# Checking again min and max of employment_duration
min_value = df_w_filters['employment_duration'].min()
max_value = df_w_filters['employment_duration'].max()
print(min_value)
print(max_value)

0.0
41.0


In [30]:
# Define the bins
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, float('inf')]

# Define labels for each bin
labels = ["0 - 5", "6 - 10", "11 - 15", "16 - 20", "21 - 25", "26 - 30","31 - 35","36 - 40","40+"]

# Create a new column with categorized income
df_w_filters['employment_duration_bucket'] = pd.cut(df_w_filters['employment_duration'], bins=bins, labels=labels, right=False)

# Print the updated DataFrame
df_w_filters.head()

<ipython-input-30-f2cdff931c02>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_filters['employment_duration_bucket'] = pd.cut(df_w_filters['employment_duration'], bins=bins, labels=labels, right=False)


,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%),age_group,employment_duration_bucket
customer_id,,,,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13,18 - 24,6 - 10
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13,25 - 34,0 - 5
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17,18 - 24,6 - 10
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5


In [35]:
# Checking again min and max of employment_duration
min_value = df_w_filters['cred_hist_length'].min()
max_value = df_w_filters['cred_hist_length'].max()
print(min_value)
print(max_value)

2
30


In [36]:
# Define the bins
bins = [0, 5, 10, 15, 20, 25, 30, float('inf')]

# Define labels for each bin
labels = ["0 - 5", "6 - 10", "11 - 15", "16 - 20", "21 - 25", "26 - 30", "30+"]

# Create a new column with categorized income
df_w_filters['cred_hist_bucket'] = pd.cut(df_w_filters['cred_hist_length'], bins=bins, labels=labels, right=False)

# Print the updated DataFrame
df_w_filters.head()

<ipython-input-36-60e83f3932bd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_filters['cred_hist_bucket'] = pd.cut(df_w_filters['cred_hist_length'], bins=bins, labels=labels, right=False)


,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%),age_group,employment_duration_bucket,cred_hist_bucket
customer_id,,,,,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13,18 - 24,6 - 10,0 - 5
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13,25 - 34,0 - 5,0 - 5
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5,0 - 5
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17,18 - 24,6 - 10,0 - 5
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5,0 - 5


In [37]:
# export the df_w_filters dataset
df_w_filters.to_csv("/content/drive/MyDrive/Colab Notebooks/Project IV/Resources/df_finalized.csv")


In [14]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project IV/Resources/df_finalized.csv", index_col="customer_id")
df.head()

,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%),age_group,employment_duration_bucket,cred_hist_bucket
customer_id,,,,,,,,,,,,,,,,,,
2.0,21,9600,OWN,5.0,EDUCATION,A,1000,11.14,1,NEW,2,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13,18 - 24,6 - 10,0 - 5
3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,5500,12.87,5,N,3,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13,25 - 34,0 - 5,0 - 5
6.0,21,9900,OWN,2.0,VENTURE,A,2500,7.14,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5,0 - 5
10.0,21,10000,OWN,6.0,VENTURE,C,1600,14.74,1,N,3,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17,18 - 24,6 - 10,0 - 5
12.0,21,10000,OWN,2.0,HOMEIMPROVEMENT,A,4500,8.63,1,N,2,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5,0 - 5


In [15]:
# remove bucketed variable

df = df.drop(["customer_age","customer_income", "loan_amnt", "loan_int_rate", "employment_duration","cred_hist_length"], axis=1)
df.head()



,home_ownership,loan_intent,loan_grade,term_years,historical_default,Current_loan_status,income_category,loan_category,Interest_Rate_Category(%),age_group,employment_duration_bucket,cred_hist_bucket
customer_id,,,,,,,,,,,,
2.0,OWN,EDUCATION,A,1,NEW,NO DEFAULT,"$0 - $20,000","$0 - $5,000",9.01 - 13,18 - 24,6 - 10,0 - 5
3.0,MORTGAGE,MEDICAL,B,5,N,DEFAULT,"$0 - $20,000","$5,001 - $10,000",9.01 - 13,25 - 34,0 - 5,0 - 5
6.0,OWN,VENTURE,A,1,N,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5,0 - 5
10.0,OWN,VENTURE,C,1,N,DEFAULT,"$0 - $20,000","$0 - $5,000",13.01 - 17,18 - 24,6 - 10,0 - 5
12.0,OWN,HOMEIMPROVEMENT,A,1,N,DEFAULT,"$0 - $20,000","$0 - $5,000",5 - 9,18 - 24,0 - 5,0 - 5


In [16]:
# print out the column datatypes

df.dtypes

,0
home_ownership,object
loan_intent,object
loan_grade,object
term_years,int64
historical_default,object
Current_loan_status,object
income_category,object
loan_category,object
Interest_Rate_Category(%),object
age_group,object


In [10]:
# Generate our categorical variable lists

df_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df_cat

['home_ownership',
 'loan_intent',
 'loan_grade',
 'historical_default',
 'Current_loan_status',
 'income_category',
 'loan_category',
 'Interest_Rate_Category(%)',
 'age_group',
 'employment_duration_bucket',
 'cred_hist_bucket']

In [18]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[df_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(df_cat)
encode_df.head()


,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,...,employment_duration_bucket_36 - 40,employment_duration_bucket_40+,employment_duration_bucket_6 - 10,cred_hist_bucket_0 - 5,cred_hist_bucket_11 - 15,cred_hist_bucket_16 - 20,cred_hist_bucket_21 - 25,cred_hist_bucket_26 - 30,cred_hist_bucket_30+,cred_hist_bucket_6 - 10
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Merge one-hot encoded features and drop the originals

df = df.merge(encode_df, left_index=True, right_index=True)
df = df.drop(df_cat, axis=1)
df.head()

,term_years,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,...,employment_duration_bucket_36 - 40,employment_duration_bucket_40+,employment_duration_bucket_6 - 10,cred_hist_bucket_0 - 5,cred_hist_bucket_11 - 15,cred_hist_bucket_16 - 20,cred_hist_bucket_21 - 25,cred_hist_bucket_26 - 30,cred_hist_bucket_30+,cred_hist_bucket_6 - 10
customer_id,,,,,,,,,,,,,,,,,,,,,
2.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Split our preprocessed data into our features and target arrays
y = df["Current_loan_status_DEFAULT"].values
X = df.drop(["Current_loan_status_DEFAULT","Current_loan_status_NO DEFAULT"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 497 (1.94 KB)

 Trainable params: 497 (1.94 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7720 - loss: 0.5660
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9448 - loss: 0.1452
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9521 - loss: 0.1136
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9552 - loss: 0.1123
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9537 - loss: 0.1111
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9564 - loss: 0.1020
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9570 - loss: 0.1034
Epoch 8/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9579 - loss: 0.0994
Epoch 9/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9586 - loss: 0.0965
Epoch 10/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9571 - loss: 0.1006
Epoch 11/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9588 - loss: 0.0977
Epoch 12/100
500/500 ━━━━━━━━━━━━━━━━━━━━

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

167/167 - 0s - 2ms/step - accuracy: 0.9587 - loss: 0.1047
Loss: 0.10474016517400742, Accuracy: 0.958700954914093
